In [1]:
%pip install gensim nltk scikit-learn

  Using cached gensim-4.3.3-cp310-cp310-macosx_11_0_arm64.whl (24.0 MB)
  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Using cached scikit_learn-1.6.1-cp310-cp310-macosx_12_0_arm64.whl (11.1 MB)
  Using cached scipy-1.13.1-cp310-cp310-macosx_12_0_arm64.whl (30.3 MB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
  Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
  Using cached click-8.1.8-py3-none-any.whl (98 kB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached regex-2024.11.6-cp310-cp310-macosx_11_0_arm64.whl (284 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Using cached wrapt-1.17.2-cp310-cp310-macosx_11_0_arm64.whl (38 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/balaji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
data = [
    ("I love this product", "Positive"),
    ("This is the best experience ever", "Positive"),
    ("I feel amazing after using this", "Positive"),
    ("I hate this product", "Negative"),
    ("This is the worst experience", "Negative"),
    ("I feel terrible using this", "Negative")
]

In [22]:
from  nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

sentences , labels = zip(*data)

tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, sg=1)

In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Function to compute sentence embeddings by averaging word vectors
# normalising labelled data.
def get_sentence_embedding(sentense_tokens, word2vec_model):
  word_vectors = [word2vec_model.wv[word] for word in sentense_tokens if word in word2vec_model.wv]
  if(len(word_vectors)> 0):
    # Axis 0, 1st Column
    return np.mean(word_vectors, axis=0)
  
  # Return a zero vector if no words are in the model
  return np.zeros(word2vec_model.vector_size)


X = np.array([get_sentence_embedding(sentence, word2vec_model) for sentence in tokenized_sentences])
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


classifier = make_pipeline(StandardScaler(), LogisticRegression())
# Train
classifier.fit(X_train, y_train)

# Test
accuracy = classifier.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.00


In [50]:
new_sentences = ["I love this pen", "This is the worst experience"]
new_tokenized = [word_tokenize(sentence.lower()) for sentence in new_sentences]
new_embeddings = np.array([get_sentence_embedding(tokens, word2vec_model) for tokens in new_tokenized])

predictions = classifier.predict(new_embeddings)
for sentence, prediction in zip(new_sentences, predictions):
    print(f"Sentence: {sentence} -> Sentiment: {prediction}")


Sentence: I love this pen -> Sentiment: Negative
Sentence: This is the worst experience -> Sentiment: Negative
